Adjusting the version of NumPy to work well with Pandas-TA

Restart the session to activate the loaded version

In [ ]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 68.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-ta 0.3.14b0 requires numpy==1.26.4, but you have numpy 1.23.5 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.

Installing Dependencies


In [ ]:
!pip install -U git+https://github.com/twopirllc/pandas-ta

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import ExtraTreesClassifier
from itertools import combinations
from collections import Counter
import seaborn as sns
import yfinance as yfin
import math
import pandas_ta as ta
from datetime import date
import time
import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = "{:,.6f}".format

  Cloning https://github.com/twopirllc/pandas-ta to /tmp/pip-req-build-vaf7gk60
  Running command git clone --filter=blob:none --quiet https://github.com/twopirllc/pandas-ta /tmp/pip-req-build-vaf7gk60
  Resolved https://github.com/twopirllc/pandas-ta to commit b465491f226d9e07fffd4e59cd0affc9284521ca
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


Loading Data - Same timeframe as the paper

In [ ]:
# Selecting the start and the end dates

start = datetime.date(2009, 12, 12)
end = datetime.date(2020, 1, 1)

# Getting the daily prices for Chile ETF

tickers = ["ECH"]

df = yfin.download(tickers, start, end, auto_adjust = False)
df.index = df.index.tz_localize('UTC')

# Flattening the MultiIndex DataFrame for the ease of Pandas-TA

if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)
df = df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

[*********************100%***********************]  1 of 1 completed


Extracting all the technical indicators based on OHLCVA of the ECH ETF

This cell takes time to run - Due to pandas version issues and the unavailability of the TA-Lib, we could extract only 112 indicators

Due to pandas version conflict, we face:

AttributeError: 'Series' object has no attribute 'append'

However, the function runs and the columns are to be found in 'df'


Continue running the next cells even after getting the error.



In [ ]:
df.ta.strategy("all")

0it [00:00, ?it/s]

[X] Please install TA-Lib to use 2crows. (pip install TA-Lib)
[X] Please install TA-Lib to use 3blackcrows. (pip install TA-Lib)
[X] Please install TA-Lib to use 3inside. (pip install TA-Lib)
[X] Please install TA-Lib to use 3linestrike. (pip install TA-Lib)
[X] Please install TA-Lib to use 3outside. (pip install TA-Lib)
[X] Please install TA-Lib to use 3starsinsouth. (pip install TA-Lib)
[X] Please install TA-Lib to use 3whitesoldiers. (pip install TA-Lib)
[X] Please install TA-Lib to use abandonedbaby. (pip install TA-Lib)
[X] Please install TA-Lib to use advanceblock. (pip install TA-Lib)
[X] Please install TA-Lib to use belthold. (pip install TA-Lib)
[X] Please install TA-Lib to use breakaway. (pip install TA-Lib)
[X] Please install TA-Lib to use closingmarubozu. (pip install TA-Lib)
[X] Please install TA-Lib to use concealbabyswall. (pip install TA-Lib)
[X] Please install TA-Lib to use counterattack. (pip install TA-Lib)
[X] Please install TA-Lib to use darkcloudcover. (pip instal

Streaming output truncated to the last 5000 lines.

  df["HA_open"][i] = 0.5 * (df["HA_open"][i - 1] + df["HA_close"][i - 1])
/usr/lib/python3.11/multiprocessing/pool.py:48: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  return list(map(*args))
/usr/lib/python3.11/multiprocessing/pool.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return list(map(*args))
/usr/local/lib/python3.11/dist-packages/pandas_ta/candles/ha.py:25: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-

AttributeError: 'Series' object has no attribute 'append'

Sanity check that 118 columns are a part of 'df'

In [ ]:
df

Price,Open,High,Low,Close,Adj Close,Volume,ABER_ZG_5_15,ABER_SG_5_15,ABER_XG_5_15,ABER_ATR_5_15,...,KURT_30,KVO_34_55_13,KVOs_34_55_13,LR_14,LOGRET_1,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,MAD_30,Gamma
Date,,,,,,,,,,,,,,,,,,,,,
2009-12-15 00:00:00+00:00,54.250000,54.580002,54.250000,54.349998,37.274147,122100,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.003490,NaN,NaN,NaN,NaN,1.000000
2009-12-16 00:00:00+00:00,55.000000,55.000000,54.299999,54.410000,37.315281,201500,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.001103,NaN,NaN,NaN,NaN,1.000000
2009-12-17 00:00:00+00:00,53.639999,54.320000,53.599998,54.209999,37.178131,72300,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.003683,NaN,NaN,NaN,NaN,-1.000000
2009-12-18 00:00:00+00:00,54.509998,54.509998,53.500000,54.389999,37.301579,104900,54.308666,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.003315,NaN,NaN,NaN,NaN,1.000000
2009-12-21 00:00:00+00:00,54.200001,54.630001,54.009998,54.599998,37.445595,54700,54.310666,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.003854,NaN,NaN,NaN,NaN,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24 00:00:00+00:00,33.549999,33.709999,33.320000,33.490002,27.117908,134300,33.606000,34.320376,32.891623,0.714377,...,-1.164325,"22,613.379749","21,344.216082",33.820989,0.000896,0.167303,0.211194,-0.043891,1.140933,-1.000000
2019-12-26 00:00:00+00:00,33.490002,33.639999,33.369999,33.570000,27.182684,295400,33.582000,34.266751,32.897248,0.684752,...,-0.985784,"28,858.395109","22,417.670229",33.789648,0.002386,0.176978,0.176696,0.000283,1.119866,-1.000000
2019-12-27 00:00:00+00:00,33.529999,33.650002,33.139999,33.459999,27.093613,209700,33.550000,34.223101,32.876898,0.673102,...,-0.663457,"23,653.795002","22,594.259482",33.714000,-0.003282,0.173767,0.138787,0.034980,1.069778,1.000000


Creating a Class Identifier as Gamma

- `Gamma = 1 if Open(t) - Open(t-1) > 0`
- `Gamma = -1 otherwise`

The first observation is returned `NaN` because of unavailability of data in `t-1` period.

Due to this reduction in order, the first row is dropped from df

In [ ]:
df["Gamma"] = np.where(df["Open"] - df["Open"].shift(1) > 0, 1,
                           np.where(df["Open"] - df["Open"].shift(1) <= 0, -1, np.nan))
df = df.iloc[1:]

Performing a Min-Max scaling of the values in the df for all the features ()

The scaling is done using the following formula:

For each observation in each feature,

`(value(i) - min)`

`----------------`

 `(max - min)`

The minimum values get the value 0

The maximum values get the value 1

In [ ]:
df_normalized = df.copy()
cols_to_normalize = df_normalized.columns.difference(['Gamma'])
df_normalized[cols_to_normalize] = df_normalized[cols_to_normalize].apply(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

Since the features include Moving Averages, there are many rows with NaN values

Features with more than 100 NaNs are removed to protect maximum data


In [ ]:
print("The number of observations in the dataframe before dropping NaNs: ", len(df_normalized))

nan_counts = df.isna().sum()

less_than_100 = (nan_counts < 100).sum()
more_or_equal_100 = (nan_counts >= 100).sum()

df_normalized = df_normalized.loc[:, df_normalized.isnull().sum() <= 100]

df_normalized = df_normalized.dropna()

print(f"Number of columns with less than 100 NaN values: {less_than_100}")
print(f"Number of columns with 100 or more NaN values: {more_or_equal_100}")
print("The number of observations in the dataframe after dropping NaNs: ", len(df_normalized))

The number of observations in the dataframe before dropping NaNs:  2528
Number of columns with less than 100 NaN values: 116
Number of columns with 100 or more NaN values: 2
The number of observations in the dataframe after dropping NaNs:  2426


Exporting df to save the dataframe. Next steps in different notebooks, will directly input this exported data.

In [ ]:
df_normalized.to_csv("/content/data.csv") # Change this directory as required.